In [20]:
!pip install chart-studio

In [21]:
#bibliotecas padrao
import numpy as np
import pandas as pd 

#webscrapping
#from bs4 import BeautifulSoup
#import requests

#plot interativo
import chart_studio.plotly as py 
import chart_studio.tools as tls 
import plotly.graph_objs as go
import cufflinks as cf

#data visualization
#import matplotlib.pyplot as plt 
#import seaborn as sb 

In [22]:
tls.set_credentials_file(username = 'taaaticorrea', api_key = 'cVyxkyjPvCNzCDggVzsn')

In [23]:
#tls.set_credentials_file(username = 'seu usuario', api_key = 'sua senha')

In [24]:
df = pd.read_csv('../input/covid19/caso.csv')
df

date state  city place_type  confirmed  deaths  \
0        2021-02-16    AC   NaN      state      53590     932   
1        2021-02-15    AC   NaN      state      53455     931   
2        2021-02-14    AC   NaN      state      53027     921   
3        2021-02-13    AC   NaN      state      52920     917   
4        2021-02-12    AC   NaN      state      52548     910   
...             ...   ...   ...        ...        ...     ...   
1489898  2020-06-23    SP  Óleo       city          1       0   
1489899  2020-06-22    SP  Óleo       city          1       0   
1489900  2020-06-21    SP  Óleo       city          1       0   
1489901  2020-06-20    SP  Óleo       city          1       0   
1489902  2020-06-19    SP  Óleo       city          1       0   

         order_for_place  is_last  estimated_population_2019  \
0                    337     True                   881935.0   
1                    336    False                   881935.0   
2                    335    False                   881935.0   
3                    334    False                   881935.0   
4                    333    False                   881935.0   
...                  ...      ...                        ...   
1489898                5    False                     2496.0   
1489899                4    False                     2496.0   
1489900                3    False                     2496.0   
1489901                2    False                     2496.0   
1489902                1    False                     2496.0   

         estimated_population  city_ibge_code  confirmed_per_100k_inhabitants  \
0                    894470.0            12.0                      5991.25739   
1                    894470.0            12.0                      5976.16466   
2                    894470.0            12.0                      5928.31509   
3                    894470.0            12.0                      5916.35270   
4                    894470.0            12.0                      5874.76383   
...                       ...             ...                             ...   
1489898                2471.0       3533809.0                        40.46945   
1489899                2471.0       3533809.0                        40.46945   
1489900                2471.0       3533809.0                        40.46945   
1489901                2471.0       3533809.0                        40.46945   
1489902                2471.0       3533809.0                        40.46945   

         death_rate  
0            0.0174  
1            0.0174  
2            0.0174  
3            0.0173  
4            0.0173  
...             ...  
1489898      0.0000  
1489899      0.0000  
1489900      0.0000  
1489901      0.0000  
1489902      0.0000  

[1489903 rows x 13 columns]

In [25]:
len(df.date.unique())

358

In [26]:
#.loc[linhas, colunas]

estados = df.loc[df.place_type == 'state', :]  #pegando apenas os estados
estados.head()

date state city place_type  confirmed  deaths  order_for_place  \
0  2021-02-16    AC  NaN      state      53590     932              337   
1  2021-02-15    AC  NaN      state      53455     931              336   
2  2021-02-14    AC  NaN      state      53027     921              335   
3  2021-02-13    AC  NaN      state      52920     917              334   
4  2021-02-12    AC  NaN      state      52548     910              333   

   is_last  estimated_population_2019  estimated_population  city_ibge_code  \
0     True                   881935.0              894470.0            12.0   
1    False                   881935.0              894470.0            12.0   
2    False                   881935.0              894470.0            12.0   
3    False                   881935.0              894470.0            12.0   
4    False                   881935.0              894470.0            12.0   

   confirmed_per_100k_inhabitants  death_rate  
0                      5991.25739      0.0174  
1                      5976.16466      0.0174  
2                      5928.31509      0.0174  
3                      5916.35270      0.0173  
4                      5874.76383      0.0173

In [27]:
estados.shape

(9104, 13)

In [28]:
todos_os_estados = list(estados.state.unique())

print(todos_os_estados)
print('Total de Estados: ',len(todos_os_estados))

['AC', 'AP', 'AM', 'DF', 'AL', 'ES', 'CE', 'MA', 'GO', 'BA', 'MS', 'MT', 'PA', 'MG', 'PI', 'PE', 'PB', 'RO', 'RJ', 'RN', 'PR', 'RR', 'SE', 'SC', 'RS', 'TO', 'SP']
Total de Estados:  27


In [29]:
data = estados[['date','state','confirmed','deaths','is_last']]  #diminuindo o dF
data.head()

date state  confirmed  deaths  is_last
0  2021-02-16    AC      53590     932     True
1  2021-02-15    AC      53455     931    False
2  2021-02-14    AC      53027     921    False
3  2021-02-13    AC      52920     917    False
4  2021-02-12    AC      52548     910    False

In [37]:
#funcao que gera o dataframe que será plotado
def desdePrimeiroCaso_estados(dados,estados, dias, situacao):
    '''
    type(dados) == DataFrame
    type(estados) == list
    type(dias) == int [0,357]
    type(situacao) == str [confirmed, deaths]
    
    '''

    covid = pd.DataFrame()

    for estado in estados:
        #pegando os estados
        df_estado = data.loc[data.state == estado, :] 
        
        #agrupando por data, somando os com mesmo dia e resetando índice
        df_estado = df_estado.groupby('date').sum()[situacao].reset_index()
            
        #dropando coluna date
        df_estado = df_estado.drop('date', axis = 1)
        df_estado.columns = [estado]
        
        #gerando dataFrame covid com os dados de interesse
        covid = pd.concat([covid, df_estado], ignore_index = False, axis = 1)
    
    covid.columns = estados
    
    return covid.head(dias)

In [31]:
df_estado = desdePrimeiroCaso_estados(df, todos_os_estados, 350, 'confirmed')
df_estado.head()

AC   AP    AM   DF   AL   ES    CE   MA   GO   BA  ...   RO    RJ   RN  \
0   3.0  1.0   2.0  1.0  1.0  1.0   9.0  1.0  3.0  1.0  ...  1.0   1.0  1.0   
1   3.0  1.0   2.0  1.0  1.0  1.0  10.0  2.0  3.0  2.0  ...  3.0   2.0  1.0   
2   4.0  1.0   3.0  1.0  1.0  1.0  19.0  2.0  4.0  2.0  ...  3.0   3.0  1.0   
3   7.0  1.0   7.0  1.0  1.0  1.0  24.0  8.0  4.0  2.0  ...  5.0  15.0  1.0   
4  11.0  1.0  11.0  2.0  1.0  1.0  68.0  8.0  9.0  3.0  ...  5.0  24.0  6.0   

     PR   RR    SE   SC   RS   TO  SP  
0   6.0  2.0   1.0  2.0  1.0  1.0   1  
1   6.0  2.0   5.0  3.0  2.0  1.0   1  
2   6.0  2.0   6.0  5.0  4.0  1.0   1  
3  12.0  8.0   7.0  6.0  4.0  2.0   2  
4  14.0  8.0  10.0  7.0  8.0  5.0   2  

[5 rows x 27 columns]

In [32]:
df_estado.iplot(filename = 'evolucao-covid-19', 
            title = 'Evolução dos casos confirmados de SarsCov-2 no Brasil por estado (by Tatiane Corrêa)',
           theme = 'pearl',
           xTitle = 'Dias após o primeiro caso',
           yTitle = 'Número de casos confirmados',
           world_readable=True)

In [33]:
df_estado = desdePrimeiroCaso_estados(df, todos_os_estados, 350, 'deaths')
df_estado.head()

AC   AP   AM   DF   AL   ES   CE   MA   GO   BA  ...   RO   RJ   RN   PR  \
0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0   
1  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0   
2  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0   
3  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0   
4  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0   

    RR   SE   SC   RS   TO  SP  
0  0.0  0.0  0.0  0.0  0.0   0  
1  0.0  0.0  0.0  0.0  0.0   0  
2  0.0  0.0  0.0  0.0  0.0   0  
3  0.0  0.0  0.0  0.0  0.0   0  
4  0.0  0.0  0.0  0.0  0.0   0  

[5 rows x 27 columns]

In [34]:
df_estado.iplot(filename = 'evolucao-covid-19 - mortes', 
            title = 'Evolução dos mortes confirmadas de SarsCov-2 no Brasil por estado (by Tatiane Corrêa)',
           theme = 'pearl',
           xTitle = 'Dias após o primeiro caso',
           yTitle = 'Número de mortes confirmadas',
           world_readable=True)

In [35]:
df_estado = desdePrimeiroCaso_estados(df, ['SP','RJ'], 350, 'deaths')
df_estado.head()

SP   RJ
0   0  0.0
1   0  0.0
2   0  0.0
3   0  0.0
4   0  0.0

In [36]:
df_estado.iplot(filename = 'evolucao-covid-SP-RJ', 
            title = 'Evolução dos mortes confirmadas de SarsCov-2 no Brasil SP/RJ (by Tatiane Corrêa)',
           theme = 'pearl',
           xTitle = 'Dias após o primeiro caso',
           yTitle = 'Número de mortes confirmadas',
           world_readable=True)

In [131]:
#funcao que gera o dataframe que será plotado
def desdePrimeiroCaso_brasil(dados,dias, situacao):
    '''
    type(dados) == DataFrame
    type(dias) == int [0,357]
    type(situacao) == str [confirmed, deaths]
    '''
    
    #pegando os estados
    df_brasil = df.loc[df.place_type == 'state' , ['date',situacao]]
       
    #agrupando por data, somando, pegando a situação e resetando índice
    df_brasil = df_brasil.groupby('date').sum()[situacao].reset_index()
            
    #dropando coluna date
    df_brasil = df_brasil.drop('date', axis = 1)
    df_brasil.columns = [situacao]
      
    #gerando dataFrame covid com os dados de interesse
    #covid = pd.concat([covid, df_estado], ignore_index = False, axis = 1) APROVEITAR DPS PARA VÁRIOS PAÍSES!
    
    #covid.columns = estados
    
    return df_brasil.head(dias)

In [132]:
df_brasil = desdePrimeiroCaso_brasil(df, 357, 'confirmed')
df_brasil.head()

confirmed
0          1
1          1
2          1
3          2
4          2

In [133]:
df_brasil.max()

confirmed    9848008
dtype: int64

In [134]:
df_brasil.iplot(filename = 'evolucao-covid-BR-casos', 
            title = 'Evolução dos casos confirmados de SarsCov-2 no Brasil (by Tatiane Corrêa)',
           theme = 'pearl',
           xTitle = 'Dias após o primeiro caso',
           yTitle = 'Número de casos confirmados',
           fill = True)

In [135]:
df_brasil = desdePrimeiroCaso_brasil(df, 357, 'deaths')
df_brasil.head()

deaths
0       0
1       0
2       0
3       0
4       0

In [ ]:
df_brasil.max()

In [121]:
df_brasil.iplot(filename = 'evolucao-covid-BR', 
            title = 'Evolução de mortes confirmadas de SarsCov-2 no Brasil (by Tatiane Corrêa)',
           theme = 'solar',
           xTitle = 'Dias após o primeiro caso',
           yTitle = 'Número de mortes confirmadas',
           fill = True)